# Noise learning helper

The error mitigation techniques [TREX](./error-mitigation-and-suppression-techniques#twirled-readout-error-extinction-trex) and [PEC](./error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec) both utilize a noise learning component based on a [Pauli-Lindblad noise model](https://www.nature.com/articles/s41567-023-02042-2) which is typically managed during execution after submitting one or more jobs through `qiskit-ibm-runtime`. However, as of `qiskit-ibm-runtime` 0.27.1 and later, a [`NoiseLearner`](../api/qiskit-ibm-runtime/noise_learner), [`LayerNoiseLearningOptions`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.options.LayerNoiseLearningOptions), and a [`MeasureNoiseLearningOptions`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.options.MeasureNoiseLearningOptions.mdx) class for executing and storing the results of these noise learning experiments now exists. This page provides an overview of its usage and the associated options available.



## Overview 

The `NoiseLearner` class performs experiments which characterizes noise processes based on a Pauli-Lindblad noise model for one (or more) circuits. It possesses a `run()` method which will execute the learning experiments and takes as input either a list of circuits or a [PUB](./primitive-input-output#overview-of-pubs) and returns a `NoiseLearnerResult` containing the learned noise channels and metadata about the job(s) submitted. Below is a code snippet demonstrating the usage of the helper program.

In [ ]:
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import Layout
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
 
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
from qiskit_ibm_runtime.noise_learner import NoiseLearner
from qiskit_ibm_runtime.options import NoiseLearnerOptions, EstimatorOptions

# Build a simple GHZ circuit with three entangling layers
circuit_to_learn = QuantumCircuit(3)
circuit_to_learn.h(0)
circuit_to_learn.cx(0, 1)
circuit_to_learn.cx(1, 2)
circuit_to_learn.cx(0, 1)

# Choose a backend to run on
service = QiskitRuntimeService()
backend = service.least_busy()

# Transpile the circuit for execution
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
circuit_to_run = pm.run(circuit_to_learn)

# Instantiate a NoiserLearner object and execute the noise learning program
learner = NoiseLearner(backend)
job = learner.run([circuit_to_run])
result = job.result()   
print(result)

The `NoiserLearnerResult.data` is a list of `LayerError` objects containing the noise model for each of the entangling layers chosen to characterize (the maximum number of the layers to learn can be specified in the `NoiserLearnerOptions`). These come in the form of one or more `LayerError` data containers which store the circuit, qubits, and `PauliLindBladError` for each layer of the noise model that was learned.

In [7]:
job = service.job("ctx5pptv0kkg008q1880")
result = job.result()

In [16]:
print(f'Noise learner result contains {len(result.data)} entries and has the following data:\n {result}\n')
print(f'The layer error model for the first layer has data:\n {result.data[0]}\n')
print(f'And the Pauli Lindblad model has has the following data: \n{result.data[0].error}\n')

Noise learner result contains 3 entries and has the following data:
 NoiseLearnerResult(data=[LayerError(circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fe0b32f2650>, qubits=[0, 1], error=PauliLindbladError(generators=['IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YI', 'YX', 'YY', 'YZ', 'ZI',
 'ZX', 'ZY', 'ZZ'], rates=[0.00037, 0.00046, 0.00035, 0.00127, 0.00067, 0.0, 0.0, 0.00067, 0.00127, 0.0, 0.0, 0.00071, 0.0, 0.00035, 0.00046]))), LayerError(circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fe0b898f190>, qubits=[0, 1, 2], error=PauliLindbladError(generators=['IIX', 'IIY', 'IIZ', 'IXI', 'IXX', 'IXY', 'IXZ', 'IYI', 'IYX', 'IYY',
 'IYZ', 'IZI', 'IZX', 'IZY', 'IZZ', 'XII', 'XXI', 'XYI', 'XZI', 'YII',
 'YXI', 'YYI', 'YZI', 'ZII', 'ZXI', 'ZYI', 'ZZI'], rates=[0.00068, 0.00023, 0.00041, 0.00093, 0.00056, 0.00034, 0.0001, 0.00023, 0.00126, 0.0, 0.0, 0.00094, 5e-05, 0.00041, 0.00023, 0.00093, 0.0, 0.0, 0.0, 0.00259, 9e-05, 9e-05, 0.0, 0.01374, 0.00024, 0.00

The `LayerError.error` attribute of the noise learning result contains the generators and error rates of the fitted Pauli Lindblad model, which has the form

$$ \Lambda(\rho) = \exp{\sum_j r_j \left(P_j \rho P_j^\dagger - \rho\right)} $$

where the $r_j$ are the `LayerError.rates` and $P_j$ are the Pauli operators specified in `LayerError.generators`.